In [ ]:
class BTreeNode:
    def __init__(self, leaf=True, t=2):
        """
        Initialize a B-Tree node
        
        Args:
            leaf (bool): True if this node is a leaf, False otherwise
            t (int): Minimum degree of the B-Tree (minimum number of keys = t-1)
        """
        self.leaf = leaf  # Is this node a leaf
        self.t = t        # Minimum degree
        self.keys = []    # List of keys
        self.children = []  # List of children references
    
    def is_full(self):
        """Check if the node has the maximum number of keys (2t-1)"""
        return len(self.keys) == 2 * self.t - 1
    
    def __str__(self):
        """String representation of node for debugging"""
        return f"Keys: {self.keys}, Leaf: {self.leaf}, Children: {len(self.children)}"


class BTree:
    def __init__(self, t=2):
        """
        Initialize an empty B-Tree
        
        Args:
            t (int): Minimum degree of the B-Tree (minimum number of keys = t-1)
        """
        self.root = BTreeNode(leaf=True, t=t)
        self.t = t  # Minimum degree
        
    def search(self, k, node=None):
        """
        Search for key k in the B-Tree
        
        Args:
            k: Key to search for
            node: Node to start the search from (defaults to the root)
            
        Returns:
            (node, index) tuple if found, None otherwise
        """
        if node is None:
            node = self.root
            
        i = 0
        # Find the first key greater than or equal to k
        while i < len(node.keys) and k > node.keys[i]:
            i += 1
        
        # If the key is found at this node
        if i < len(node.keys) and k == node.keys[i]:
            return (node, i)
        
        # If this is a leaf node and the key is not found
        if node.leaf:
            return None
        
        # Recursively search in the appropriate child
        return self.search(k, node.children[i])
    
    def insert(self, k):
        """
        Insert key k into the B-Tree
        
        Args:
            k: Key to insert
        """
        root = self.root
        
        # If the root is full, the tree grows in height
        if root.is_full():
            # Create a new root
            new_root = BTreeNode(leaf=False, t=self.t)
            new_root.children.append(root)
            self.root = new_root
            
            # Split the old root
            self._split_child(new_root, 0)
            
            # Insert the key into the new root
            self._insert_non_full(new_root, k)
        else:
            # Insert the key into the non-full root
            self._insert_non_full(root, k)
    
    def _split_child(self, parent, index):
        """
        Split the child of parent at index
        
        Args:
            parent: Parent node
            index: Index of the child to split
        """
        t = self.t
        child = parent.children[index]
        
        # Create a new node which will be a sibling of child
        new_child = BTreeNode(leaf=child.leaf, t=t)
        
        # Move the keys from child to new_child
        new_child.keys = child.keys[t:]
        child.keys = child.keys[:t-1]
        
        # If not a leaf, move the children as well
        if not child.leaf:
            new_child.children = child.children[t:]
            child.children = child.children[:t]
        
        # Insert the new child into the parent
        parent.children.insert(index + 1, new_child)
        
        # Move the middle key to the parent
        middle_key = child.keys[t-1]
        parent.keys.insert(index, middle_key)
    
    def _insert_non_full(self, node, k):
        """
        Insert key k into a non-full node
        
        Args:
            node: Node to insert into
            k: Key to insert
        """
        i = len(node.keys) - 1
        
        # If this is a leaf node
        if node.leaf:
            # Find the location to insert the key
            while i >= 0 and k < node.keys[i]:
                i -= 1
            
            # Insert the key
            node.keys.insert(i + 1, k)
        else:
            # Find the child which will have the key
            while i >= 0 and k < node.keys[i]:
                i -= 1
            i += 1
            
            # If the child is full, split it
            if node.children[i].is_full():
                self._split_child(node, i)
                
                # After splitting, determine which child to go into
                if k > node.keys[i]:
                    i += 1
            
            # Insert the key into the child
            self._insert_non_full(node.children[i], k)
    
    def delete(self, k):
        """
        Delete key k from the B-Tree
        
        Args:
            k: Key to delete
        """
        if not self.root.keys:
            return  # Empty tree
        
        self._delete_key(self.root, k)
        
        # If the root has no keys and has a child, make the child the new root
        if not self.root.keys and not self.root.leaf:
            self.root = self.root.children[0]
    
    def _delete_key(self, node, k):
        """
        Delete key k from the given node
        
        Args:
            node: Node to delete from
            k: Key to delete
        """
        t = self.t
        
        # Find the position of the key
        i = 0
        while i < len(node.keys) and k > node.keys[i]:
            i += 1
        
        # If the key is in this node
        if i < len(node.keys) and node.keys[i] == k:
            # Case 1: If this is a leaf node, simply remove the key
            if node.leaf:
                node.keys.pop(i)
                return
            
            # Case 2: If the key is not in a leaf node
            # Case 2a: If the child preceding k has at least t keys
            if len(node.children[i].keys) >= t:
                # Find the predecessor
                pred_node = node.children[i]
                while not pred_node.leaf:
                    pred_node = pred_node.children[-1]
                
                # Replace k with the predecessor
                pred_key = pred_node.keys[-1]
                node.keys[i] = pred_key
                
                # Recursively delete the predecessor
                self._delete_key(node.children[i], pred_key)
                
            # Case 2b: If the child following k has at least t keys
            elif len(node.children[i+1].keys) >= t:
                # Find the successor
                succ_node = node.children[i+1]
                while not succ_node.leaf:
                    succ_node = succ_node.children[0]
                
                # Replace k with the successor
                succ_key = succ_node.keys[0]
                node.keys[i] = succ_key
                
                # Recursively delete the successor
                self._delete_key(node.children[i+1], succ_key)
                
            # Case 2c: If both preceding and following children have t-1 keys
            else:
                # Merge k and node.children[i+1] into node.children[i]
                child = node.children[i]
                child.keys.append(node.keys.pop(i))
                
                # Merge the children
                next_child = node.children.pop(i+1)
                child.keys.extend(next_child.keys)
                
                if not child.leaf:
                    child.children.extend(next_child.children)
                
                # Recursively delete k from the merged node
                self._delete_key(child, k)
                
        # If the key is not in this node
        else:
            # If this is a leaf node, the key doesn't exist in the tree
            if node.leaf:
                return
            
            # Determine whether the key is in the last child
            last_child = (i == len(node.keys))
            
            # If the child has t-1 keys, fill it
            if len(node.children[i].keys) == t - 1:
                self._fill_child(node, i)
                
                # After filling, the child might have changed
                if last_child and i > len(node.keys):
                    # Recursively delete the key from the previous child
                    self._delete_key(node.children[i-1], k)
                else:
                    # Recursively delete the key from the current child
                    self._delete_key(node.children[i], k)
            else:
                # Recursively delete the key from the current child
                self._delete_key(node.children[i], k)
    
    def _fill_child(self, node, index):
        """
        Fill a child that has t-1 keys
        
        Args:
            node: Parent node
            index: Index of the child to fill
        """
        t = self.t
        
        # Case 1: Borrow from the left sibling
        if index > 0 and len(node.children[index-1].keys) >= t:
            self._borrow_from_prev(node, index)
            
        # Case 2: Borrow from the right sibling
        elif index < len(node.keys) and len(node.children[index+1].keys) >= t:
            self._borrow_from_next(node, index)
            
        # Case 3: Merge with a sibling
        else:
            if index != len(node.keys):
                self._merge(node, index)
            else:
                self._merge(node, index-1)
    
    def _borrow_from_prev(self, node, index):
        """
        Borrow a key from the previous sibling
        
        Args:
            node: Parent node
            index: Index of the child
        """
        child = node.children[index]
        sibling = node.children[index-1]
        
        # Move a key from parent to child
        child.keys.insert(0, node.keys[index-1])
        
        # Move a key from sibling to parent
        node.keys[index-1] = sibling.keys.pop()
        
        # If both are internal nodes, move the last child of sibling to child
        if not child.leaf:
            child.children.insert(0, sibling.children.pop())
    
    def _borrow_from_next(self, node, index):
        """
        Borrow a key from the next sibling
        
        Args:
            node: Parent node
            index: Index of the child
        """
        child = node.children[index]
        sibling = node.children[index+1]
        
        # Move a key from parent to child
        child.keys.append(node.keys[index])
        
        # Move a key from sibling to parent
        node.keys[index] = sibling.keys.pop(0)
        
        # If both are internal nodes, move the first child of sibling to child
        if not sibling.leaf:
            child.children.append(sibling.children.pop(0))
    
    def _merge(self, node, index):
        """
        Merge the child at index with the child at index+1
        
        Args:
            node: Parent node
            index: Index of the first child
        """
        child = node.children[index]
        sibling = node.children[index+1]
        
        # Move a key from parent to child
        child.keys.append(node.keys.pop(index))
        
        # Copy all keys from sibling to child
        child.keys.extend(sibling.keys)
        
        # Copy the children if not leaf nodes
        if not child.leaf:
            child.children.extend(sibling.children)
        
        # Remove the sibling
        node.children.pop(index+1)
    
    def inorder_traversal(self, node=None):
        """
        Perform an inorder traversal of the B-Tree
        
        Args:
            node: Node to start the traversal from (defaults to the root)
            
        Returns:
            List of keys in sorted order
        """
        result = []
        if node is None:
            node = self.root
        
        # Recursively visit the subtrees
        for i in range(len(node.keys)):
            if not node.leaf:
                result.extend(self.inorder_traversal(node.children[i]))
            result.append(node.keys[i])
        
        # Visit the last subtree
        if not node.leaf:
            result.extend(self.inorder_traversal(node.children[-1]))
        
        return result
    
    def range_search(self, start, end):
        """
        Return all keys in the range [start, end]
        
        Args:
            start: Start of the range
            end: End of the range
            
        Returns:
            List of keys in the range
        """
        result = []
        self._range_search_helper(self.root, start, end, result)
        return result
    
    def _range_search_helper(self, node, start, end, result):
        """
        Helper function for range search
        
        Args:
            node: Current node
            start: Start of the range
            end: End of the range
            result: List to collect the results
        """
        # Find the first key >= start
        i = 0
        while i < len(node.keys) and start > node.keys[i]:
            i += 1
        
        # Traverse the tree
        if not node.leaf:
            # Search in the left subtree if needed
            self._range_search_helper(node.children[i], start, end, result)
        
        # Add keys that are in the range
        while i < len(node.keys) and node.keys[i] <= end:
            result.append(node.keys[i])
            i += 1
            
            # Search in the middle subtrees if not a leaf
            if not node.leaf and i < len(node.children):
                self._range_search_helper(node.children[i], start, end, result)
    
    def visualize(self):
        """
        Return a representation of the B-Tree for visualization
        
        Returns:
            Dictionary containing nodes and edges for visualization
        """
        nodes = {}
        edges = []
        self._visualize_helper(self.root, 0, 0, nodes, edges)
        return {'nodes': nodes, 'edges': edges}
    
    def _visualize_helper(self, node, node_id, parent_id, nodes, edges, is_root=True):
        """
        Helper function for visualization
        
        Args:
            node: Current node
            node_id: ID of the current node
            parent_id: ID of the parent node
            nodes: Dictionary to collect node information
            edges: List to collect edge information
            is_root: Whether this node is the root
        """
        # Add the node
        nodes[node_id] = {
            'keys': node.keys,
            'leaf': node.leaf
        }
        
        # Add edge from parent to this node (except for root)
        if not is_root:
            edges.append((parent_id, node_id))
        
        # Process children
        next_id = node_id + 1
        for i, child in enumerate(node.children):
            edges.append((node_id, next_id))
            next_id = self._visualize_helper(child, next_id, node_id, nodes, edges, False)
        
        return next_id